In [1]:
import pandas as pd
import numpy as np
import numpy as np
import os
from tensorflow.keras.applications import VGG19, InceptionV3
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.utils import to_categorical
import numpy as np


In [2]:
path_test_ENDO = 'dados/endo'
path_test_OUTRO = 'dados/outros'

# Lista de todas as imagens
list_img_endo = os.listdir(path_test_ENDO)
list_img_outros = os.listdir(path_test_OUTRO)

x_data = []
y_data = []

# Lendo as imagens que tem endo
for img_name in list_img_endo:
    img_path = os.path.join(path_test_ENDO, img_name)
    img = image.load_img(img_path, target_size=(512, 512))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    x_data.append(img_array)
    y_data.append(0)  # Rótulo 0 para NORMAL

# Lendo as imagens que tem outros
for img_name in list_img_outros:
    img_path = os.path.join(path_test_OUTRO, img_name)
    img = image.load_img(img_path, target_size=(512, 512))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    x_data.append(img_array)
    y_data.append(1)  # Rótulo 1 para PNEUMONIA

x_data = np.array(x_data)
y_data = np.array(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of x_train: (92, 512, 512, 3)
Shape of x_test: (24, 512, 512, 3)
Shape of y_train: (92,)
Shape of y_test: (24,)


In [3]:
num_classes = 2
#VGG19 pré-treinado
#VGG19 sem as camadas totalmente conectadas
base_model = VGG19(weights='imagenet', include_top=False)

#camadas totalmente conectadas para a nova tarefa de classificação
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#Novo modelo combinando a base pré-treinada com as camadas personalizadas
model = Model(inputs=base_model.input, outputs=predictions)

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-12-19 15:42:33.913825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 15:42:33.915946: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Tranformando os rótulos em codificação one-hot
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Treinando o modelo
model.fit(x_train, y_train_one_hot, epochs=10, batch_size=2, validation_data=(x_test, y_test_one_hot))

# Medindo o desempenho do modelo nos dados de teste
accuracy = model.evaluate(x_test, y_test_one_hot)[1]
print(f'Acurácia nos dados de teste: {accuracy * 100:.2f}%')

Epoch 1/10


2023-12-19 15:42:34.908025: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-19 15:42:35.207580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - ETA: 0s - loss: 1.2790 - accuracy: 0.5109

2023-12-19 15:42:46.576744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - 14s 301ms/step - loss: 1.2790 - accuracy: 0.5109 - val_loss: 0.6249 - val_accuracy: 0.7500
Epoch 2/10
46/46 [==============================] - 14s 302ms/step - loss: 0.7342 - accuracy: 0.5652 - val_loss: 0.6702 - val_accuracy: 0.5833
Epoch 3/10
46/46 [==============================] - 14s 297ms/step - loss: 0.6239 - accuracy: 0.6848 - val_loss: 0.6572 - val_accuracy: 0.7083
Epoch 4/10
46/46 [==============================] - 14s 295ms/step - loss: 0.7122 - accuracy: 0.6739 - val_loss: 0.6905 - val_accuracy: 0.5833
Epoch 5/10
46/46 [==============================] - 14s 301ms/step - loss: 0.6106 - accuracy: 0.6522 - val_loss: 0.5731 - val_accuracy: 0.6667
Epoch 6/10
46/46 [==============================] - 14s 299ms/step - loss: 0.5141 - accuracy: 0.7935 - val_loss: 0.6271 - val_accuracy: 0.6250
Epoch 7/10
46/46 [==============================] - 14s 301ms/step - loss: 0.4482 - accuracy: 0.8261 - val_loss: 0.9732 - val_accuracy: 0.6250
Epoch 8/10

2023-12-19 15:44:55.946465: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 9s 9s/step - loss: 0.9936 - accuracy: 0.7083
Acurácia nos dados de teste: 70.83%


In [5]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Medindo o desempenho do modelo nos dados de teste
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)

# Métricas
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Matriz de confusão
conf_matrix = confusion_matrix(y_true, y_pred_classes)

print(f'Acurácia nos dados de teste: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('\nMatriz de Confusão:')
print(conf_matrix)


2023-12-19 15:45:05.958467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
Acurácia nos dados de teste: 70.83%
Precision: 0.7325
Recall: 0.7083
F1 Score: 0.6818

Matriz de Confusão:
[[13  1]
 [ 6  4]]
